# Importing Modules

In [ ]:
import time
start = time.time()

Import the time module and create a variable named "start". The start variable will be used later to track the runtime of operations.

In [ ]:
import datetime
now = datetime.datetime.now()
curTime = now.strftime("%H:%M:%S")
print("Start Time: ", curTime)

Use datetime to get the current time, used to display when the script has started running. Useful for calculating when an operation will end.

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import io
from sklearn.preprocessing import OneHotEncoder

Import the remaining modules. The modules are used to the following:<br>
-Pandas: Used for the dataframe datatype which is easy to operate on as well as convert to other files. It is also compatable with fitting for many ML models.<br>
-Numpy: Used for various calculations and time conversions.<br>
-SQLAlchemy: Used to import data from database and convert to a dataframe.<br>
-io: Used to convert ANSI csv files to UTF-8 in order to make special characters readable.<br>
-OneHotEncoder: Needed to convert categorical variables into dummy variables.<br>

# Getting and converting the data

In [ ]:
runTimeBool = True

When set to true, messages will be printed to the console detailing the runtimes at each checkpoint.

In [ ]:
if runTimeBool == True:
    checkpoint = round(time.time()-start, 2)
    print(f"Module import runtime: {checkpoint}")

Calculate the runtime of the script to that point, remember to round the float in the checkpoint operation in order to improve runtime preformance.

In [ ]:
getDataFromSQL = True
if getDataFromSQL == True:
    
    server = "hpeanneops.rose.rdlabs.hpecorp.net"
    username = "ops_aruba_writer"
    password = "W0nderfu1Pa55w0rd"
    db = "NEO"
    driver = "ODBC Driver 17 for SQL Server"

    dbconnection = f"mssql://{username}:{password}@{server}/{db}?driver={driver}"
    engine = sqla.create_engine(dbconnection)
    connection = engine.connect()

    sqltopandasdf = pd.read_sql("SELECT	R.ReqMasterId, R.EMPType, R.ReqStatus, R.NbofReqs, R.HiringManagerName, R.HiringManagerId, \
    R.REQApprovalDate, R.CostCenter, R.City, R.Country, R.JobFamily, R.JobCode, \
    (SELECT TOP  1 P.[month] + '-01' FROM HeadcountPlan P WHERE P.ReqNumber = R.ReqNumber ORDER BY P.[month] desc) AS StartDate \
    FROM	RequisitionMaster R \
    WHERE	(SELECT TOP  1 P.[month] + '-01' FROM HeadcountPlan P WHERE P.ReqNumber = R.ReqNumber ORDER BY P.[month] desc) IS NOT NULL \
    and R.ReqStatus <> 'Closed' and R.EMPType <> 'INT'", engine)
    
    sqltopandasdf.to_csv("compiledata.csv", index=False)
    
    if runTimeBool == True:
        checkpoint = round(time.time()-start, 2)
        print(f"Database import runtime: {checkpoint}")

When getDataFromSQL is set to True, the above code will use the input credential to access the NEO database and create an engine to read the database then convert it to a dataframe. The server, username, password, database, and SQL command can all be easily changed. The final result from the SQL pull will be exported to a csv. When getDataFromSQL is set to False, the script will try to pull data from an existing compiledata.csv 

In [ ]:
initialcsv = "compiledata.csv"
try:
    with io.open("compiledata.csv", encoding="ANSI") as outer:
        with io.open("convertedcompileddata.csv", "w", encoding="UTF-8") as inner:
            for line in outer:
                inner.write(line)
except:
    dummydf = pd.read_csv("convertedcompileddata.csv", index_col=False)
    dummydf.to_csv("convertedcompileddata.csv", index=False)

Try to convert the data obtained from ANSI into UTF-8 encoding to avoid reading errors with pandas. End result is exported to csv. 

# Preprocessing

In [ ]:
def findDupeCols(pandasdf):
    dupeCols = set()
    for x in range(pandasdf.shape[1]):
        col = pandasdf.iloc[:,x]
        for y in range(x+1,pandasdf.shape[1]):
            col2 = pandasdf.iloc[:,y]
            if col.equals(col2):
                dupeCols.add(pandasdf.columns.values[x])
    return list(dupeCols)
dupecols = findDupeCols(pandasdf)
pandasdf.drop(columns = dupecols, inplace=True)

Fucntion to loop through the values in each column and compare them to each other column. If the columns have identical values, then the columns will be dropped. This is to prevent overfitting.

In [ ]:
for index, row in pandasdf.iterrows():
    if row["ReqStatus"].lower() == "closed":
        pandasdf.drop(index, axis=0, inplace=True)
pandasdf = pandasdf.reset_index(drop=True)

Iterate over the dataframe and if the ReqStatus column is marked as "closed", drop the row. Currently made redundant by the initial SQL command. Mind the dataframe reset_index function, as it is required every time a row is dropped from a dataframe otherwise indexing errors will occur.

In [ ]:
clearCompleteDupeReqs = True
alsoClearUncompleteDupes = False
if clearCompleteDupeReqs == True:
    dupeRecList = []
    ucDupeRecList = []
    for index, row in pandasdf.iterrows():
        if row["NbofReqs"] > 1:
            concatRow = str(row["NbofReqs"]) + str(row["City"]) + str(row["HiringManagerName"] + str(row["REQApprovalDate"]))
            if row["ReqStatus"] == "Filled" or row["ReqStatus"] == "Filled in WD with open status":
                if concatRow in dupeRecList:
                    pandasdf.drop(index, inplace=True)
                else:
                    dupeRecList.append(concatRow)
            else:
                if alsoClearUncompleteDupes == True:
                    if concatRow in ucDupeRecList:
                        pandasdf.drop(index, inplace=True)
                    else:
                        ucDupeRecList.append(concatRow)
                else:
                    pass
pandasdf.reset_index()

Check for reqs with the same NbofReqs, City, and HiringManager and drop them if they are the same in order to avoid overfitting and certain reqs affecting the model and target accuracy more than they should. Can change the two vaiables clearCompleteDupeReqs and alsoClearUncompleteDupes to change this.

In [ ]:
if getDataFromSQL == True:
    pandasdf["REQApprovalDate"] = pd.to_datetime(pandasdf["REQApprovalDate"], format="%Y-%m-%d")
    pandasdf["StartDate"] = pd.to_datetime(pandasdf["StartDate"], format="%Y-%m-%d")
else:
    pandasdf["REQApprovalDate"] = pd.to_datetime(pandasdf["REQApprovalDate"], format="%m/%d/%Y")
    pandasdf["StartDate"] = pd.to_datetime(pandasdf["StartDate"], format="%m/%d/%Y")

Convert the dates pulled from SQL into pandas datetime which is operable by numpy. The conversion depends on the source, the dates pulled from SQL server will use the Y-m-d format while csv files use the m/d/Y format.

In [ ]:
ununiqueCols = [x for x in pandasdf.columns if pandasdf[x].nunique()==1]
pandasdf.drop(ununiqueCols, axis=1, inplace=True)

In [ ]:
edgedate = "1905-01-01 00:00:00"
edgedate64 = np.datetime64(edgedate)
for date in pandasdf["StartDate"]:
    if date < edgedate64:
        pandasdf.drop(pandasdf.index[(pandasdf["StartDate"] == date)], axis=0, inplace=True)
for date in pandasdf["REQApprovalDate"]:
    if date < edgedate64:
        pandasdf.drop(pandasdf.index[(pandasdf["REQApprovalDate"] == date)], axis=0, inplace=True)
pandasdf = pandasdf.reset_index(drop=True)

Drop any rows with a StartDate or REQApprovalDate that is before Janurary 1st 1905. This date is arbitrary and is intended to cull any unreliable data using just the database default date (1901-01-01) for either StartDate or REQApprovalDate. This date could be changed to anything.

In [ ]:
yTestStartDates = []
for index, row in pandasdf.iterrows():
    if row["ReqStatus"].lower() == "open" or row["ReqStatus"].lower() == "in_progress" or row["ReqStatus"] == "Frozen":
        yTestStartDates.append((row["StartDate"]-row["REQApprovalDate"]).days)
        pandasdf.loc[index, "StartDate"] = None

Get the age for the incomplete reqs based on Fred's predicted values, used for comparisons.

In [ ]:
nameList = []
for index, row in pandasdf.iterrows():
    if row["HiringManagerId"] == "On Leave" and row["HiringManagerName"] not in nameList:
        nameList.append(row["HiringManagerName"])
idDict = {}
for index, row in pandasdf.iterrows():
    if row["HiringManagerName"] in nameList and row["HiringManagerId"].lower() != "on leave":
        idDict[row["HiringManagerName"]] = row["HiringManagerId"]
for index, row in pandasdf.iterrows():
    if row["HiringManagerName"] in idDict.keys() and row["HiringManagerId"].lower() == "on leave":
        row["HiringManagerId"] = idDict.get(str(row["HiringManagerName"]))

First, find reqs where the HiringManagerId is erroneously labeled as "On Leave" and store those in a list. Then, check if those names have another req ID associated with them. If they do, store the name and the ID in a dictionary with the name being the key and the ID being the value. Then iterate over the dataframe one last time and replace the IDs that say "On Leave" with the actual ID associated with the name. 

In [ ]:
createYearList = []
createMonthList = []
createDayList = []
startYearList = []
startMonthList = []
startDayList = []
for index, row in pandasdf.iterrows():
    createYearList.append(row["REQApprovalDate"].year)
    createMonthList.append(row["REQApprovalDate"].month)
    #createDayList.append(row["REQApprovalDate"].day)
    startYearList.append(row["StartDate"].year)
    startMonthList.append(row["StartDate"].month)
    #startDayList.append(row["StartDate"].day)
pandasdf.insert(pandasdf.shape[1], "creation year", createYearList)
pandasdf.insert(pandasdf.shape[1], "creation month", createMonthList)
#pandasdf.insert(pandasdf.shape[1], "creation day", createDayList)
pandasdf.insert(pandasdf.shape[1], "start year", startYearList)
pandasdf.insert(pandasdf.shape[1], "start month", startMonthList)
#pandasdf.insert(pandasdf.shape[1], "start day", startDayList)

if runTimeBool == True:
    checkpoint = round(time.time()-start, 2)
    print(f"Editing conversion Runtime: {checkpoint}")

Convert the days, months, and years in StartDate and REQApprovalDate to their own columns. The operations converting creation and start day are commented out as they are currently not being used on the model as they will have a disproportionate amount of weight on the model and the results are sorted in 15 day buckets. Currently, none of these factors are being used in the model as they were found to be detrimental to the overall results due to causing overfitting. 

In [ ]:
reqMasterIDList = []
removeReqMasterId = True
if removeReqMasterId == True:
    reqMasterIDList  = pandasdf["ReqMasterId"].tolist()
    pandasdf.drop("ReqMasterId", axis=1)

Store the values in the ReqMasterId column in a list and drop the ReqMasterId column in the dataframe.

In [ ]:
oheBool = True
if oheBool == True:
    ohe = OneHotEncoder()
    #pandasdf["start year"] = (pandasdf["start year"].astype("category")).cat.codes
    #pandasdf["start month"] = (pandasdf["start month"].astype("category")).cat.codes
    #pandasdf["ReqStatus"] = (pandasdf["ReqStatus"].astype("category")).cat.codes
    pandasdf["creation month"] = (pandasdf["creation month"].astype("category")).cat.codes
    pandasdf["HiringManagerId"] = (pandasdf["HiringManagerId"].astype("category")).cat.codes
    pandasdf["CostCenter"] = (pandasdf["CostCenter"].astype("category")).cat.codes
    pandasdf["City"] = (pandasdf["City"].astype("category")).cat.codes
    pandasdf["Country"] = (pandasdf["Country"].astype("category")).cat.codes
    #pandasdf["JobFamily"] = (pandasdf["JobFamily"].astype("category")).cat.codes
    pandasdf["JobCode"] = (pandasdf["JobCode"].astype("category")).cat.codes

    ohearray = pd.DataFrame(ohe.fit_transform(pandasdf[[#"creation month", #"ReqStatus",
            "HiringManagerId", "CostCenter", "City",
            "Country", "JobCode"]]).toarray())
    pandasdf = pandasdf.join(ohearray)
    pandasdf = pandasdf.drop(["creation month", #"start month", #"ReqStatus",
            "HiringManagerName", "NbofReqs", "HiringManagerId", "CostCenter", "City",
            "Country", "JobCode"], axis=1)

if runTimeBool == True:
    checkpoint = round(time.time()-start, 2)
    print(f"Encoding runtime: {checkpoint}")

OHE stands for One Hot Encoder, an encoding method that converts all categorical variables to their own binary columns. Often times required by most models to read categorical variables. First, convert the columns of the dataframe into categories, then convert the category columns into encoded dummy columns and attach them to the dataframe. Drop the original categorical columns. In this current iteration, the start year, start month, ReqStatus, and JobFamily conversions to category datatype are commented out and those columns are not encoded. For start month and start year, as previously mentioned, they are not included in the model because they hurt the model due to overfitting. ReqStatus is dropped because the ReqStatus of the predicted values cannot match the ReqStatus of the actual values. JobFamily is not used because it has been replaced by JobCode, which is more diverse and nuanced, as it includes the requirement of the position as well as the job type and tends to lead to more accurate results on a macro level.

In [ ]:
testData = pandasdf[pandasdf.StartDate.isnull()].reset_index(drop=True)
testData.insert(testData.shape[1], "Age", yTestStartDates)
ageList = []
for index, row in pandasdf.iterrows():
    age = (row["StartDate"] - row["REQApprovalDate"]).days
    if age >= 0:
        ageList.append(age)
    else:
        pandasdf = pandasdf.drop(index=index)
pandasdf.insert(pandasdf.shape[1], "Age", ageList)
pandasdf = pandasdf.dropna(axis=0, subset=["StartDate"])
pandasdf = pandasdf.reset_index(drop=True)

First, copy the rows that do not have a StartDate to its own testData dataframe and drop those rows in the original dataframe. Then, attach the predicted values to the testData dataframe. Iterate over the original dataframe and subtract the StartDate from the REQApprovalDate to determine the age of the req. Reqs with a negative age (StartDate is incorrectly input before REQApprovalDate) are culled. This operation that calculates eats up the majority of the runtime for this script with more categorical variables and more rows. If possible, try to optimize to only consider the two columns it needs to calculate on rather than iterating over the entire dataframe.

In [ ]:
bucketList = []
if pandasdf["Age"].max() > testData["Age"].max():
    for i in range(0, pandasdf["Age"].max()+15, 15):
        bucketList.append(i)
else:
    for i in range(0, testData["Age"].max()+15, 15):
        bucketList.append(i)

pandasdf["AgeGroups"] = pd.cut(pandasdf["Age"], bucketList, include_lowest=True)
testData["AgeGroups"] = pd.cut(testData["Age"], bucketList, include_lowest=True)

After calculating age, create a list of 15 day buckets representing every possible outcome. Then, use that list in pandas' cut function which automatically sorts each value into their respective buckets. These categorical buckets are now the targets that the model will predict on going forward.

# Final Exporting

In [ ]:
xaxis = pandasdf.drop(["Age", "StartDate", "start year", "REQApprovalDate"], axis=1)
yaxis = pandasdf["Age"]
xtest = testData.drop(["Age", "StartDate", "start year", "REQApprovalDate"], axis=1)
ytest = testData["Age"]

Create new dataframes that are split between the variables and results.

In [ ]:
if oheBool == True:
    xtest.columns = xtest.columns.astype(str)
    xaxis.columns = xaxis.columns.astype(str)

Convert the column names to string to avoid an error.

In [ ]:
exportAxisAndTest = True
if exportAxisAndTest == True:
    xaxis.to_csv("xaxis.csv", index=False, encoding="utf-8")
    xtest.to_csv("xtest.csv", index=False, encoding="utf-8")
    yaxis.to_csv("yaxis.csv", index=False, encoding="utf-8")
    ytest.to_csv("ytest.csv", index=False, encoding="utf-8")
    
    if runTimeBool == True:
        checkpoint = round(time.time()-start, 2)
        print(f"Export Test and Axis: {checkpoint}\n")

Export dataframes to csv. These csvs will be called on and used in other scripts.

In [ ]:
print("Finish")